In [117]:
%matplotlib inline
import numpy as np
import pandas as pd
import cufflinks as cf  
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
cf.go_offline()

In [118]:
liver_patient = pd.read_csv("duomenys/Indian_Liver_Patient.csv")
liver_patient.head()

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1


In [119]:
#pervadiname stulpelį dataset į liver_disease
liver_patient.rename(columns={'Dataset': 'Liver_Disease'}, inplace=True)
liver_patient['Gender'] = liver_patient['Gender'].apply(lambda x: 1 if x =='Male' else 0)
#Female=0 and Male = 1
liver_patient['Albumin_and_Globulin_Ratio'].mean()
liver_patient=liver_patient.fillna(0.94)

In [120]:
liver_patient.groupby('Liver_Disease').size()

Liver_Disease
1    416
2    167
dtype: int64

In [121]:
liver_patient.columns # Pasižiūrime visų stulpelių pavadinimus

Index(['Age', 'Gender', 'Total_Bilirubin', 'Direct_Bilirubin',
       'Alkaline_Phosphotase', 'Alamine_Aminotransferase',
       'Aspartate_Aminotransferase', 'Total_Protiens', 'Albumin',
       'Albumin_and_Globulin_Ratio', 'Liver_Disease'],
      dtype='object')

In [122]:
# Pasidarome X ir Y, kurį prognozuosime
X = liver_patient[['Age', 'Gender', 'Total_Bilirubin', 'Direct_Bilirubin',
       'Alkaline_Phosphotase', 'Alamine_Aminotransferase',
       'Aspartate_Aminotransferase', 'Total_Protiens', 'Albumin',
       'Albumin_and_Globulin_Ratio']]
Y = liver_patient['Liver_Disease']

In [123]:
# Pasidarome test ir train rinkinius
np.random.seed(1)
mask = np.random.random(liver_patient.shape[0]) < 0.7

PIRMAS MODELIS

In [124]:
model = LogisticRegression()

model.fit(liver_patient[mask].drop('Liver_Disease', axis=1), liver_patient[mask]['Liver_Disease'])
pred = model.predict(liver_patient[~mask].drop('Liver_Disease', axis=1))
prob = model.predict_proba(liver_patient[~mask].drop('Liver_Disease', axis=1))

C:\Users\nasev\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.



In [125]:
print('Log-loss = {0:.02f}'.format(
    log_loss(liver_patient[~mask]['Liver_Disease'], prob)))
print('Hit rate = {0:.02%}'.format(
    (liver_patient[~mask]['Liver_Disease'] == pred).mean()))

Log-loss = 0.52
Hit rate = 70.79%


In [126]:
pd.DataFrame(confusion_matrix(liver_patient[~mask]['Liver_Disease'], pred),
             columns=model.classes_, index=model.classes_)

,1,2
1,116,7
2,45,10


ANTRAS MODELIS 
(Pridėjus naujus stulpelius)


In [127]:
#Apsiskaiciuojame Globulina
liver_patient['Globulin'] = liver_patient['Albumin'] / liver_patient['Albumin_and_Globulin_Ratio']

#Susiskaiciuojam netiesiogini bilirubina (nekonjuguotas) (jis lygus is bendro atemus tiesiogini)
liver_patient['Indirect_Bilirubin'] = liver_patient['Total_Bilirubin'] - liver_patient['Direct_Bilirubin']
liver_patient['Protiens_Ratio'] = liver_patient['Total_Protiens'] / liver_patient['Albumin']

liver_patient.head()


,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Liver_Disease,Globulin,Indirect_Bilirubin,Protiens_Ratio
0,65,0,0.7,0.1,187,16,18,6.8,3.3,0.90,1,3.666667,0.6,2.060606
1,62,1,10.9,5.5,699,64,100,7.5,3.2,0.74,1,4.324324,5.4,2.343750
2,62,1,7.3,4.1,490,60,68,7.0,3.3,0.89,1,3.707865,3.2,2.121212
3,58,1,1.0,0.4,182,14,20,6.8,3.4,1.00,1,3.400000,0.6,2.000000
4,72,1,3.9,2.0,195,27,59,7.3,2.4,0.40,1,6.000000,1.9,3.041667


In [128]:
np.random.seed(1)
mask = np.random.random(liver_patient.shape[0]) < 0.7

In [129]:
model2 = LogisticRegression()

model2.fit(liver_patient[mask].drop('Liver_Disease', axis=1), liver_patient[mask]['Liver_Disease'])
pred2 = model2.predict(liver_patient[~mask].drop('Liver_Disease', axis=1))
prob2 = model2.predict_proba(liver_patient[~mask].drop('Liver_Disease', axis=1))

print('Log-loss = {0:.02f}'.format(
    log_loss(liver_patient[~mask]['Liver_Disease'], prob2)))
print('Hit rate = {0:.02%}'.format(
    (liver_patient[~mask]['Liver_Disease'] == pred2).mean()))

Log-loss = 0.52
Hit rate = 71.35%


C:\Users\nasev\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.



In [130]:
# suskaičiuojame Cofusion matrix
pd.DataFrame(confusion_matrix(liver_patient[~mask]['Liver_Disease'], pred2),
             columns=model2.classes_, index=model2.classes_)

,1,2
1,115,8
2,43,12


TREČIAS MODELIS (išmetus stulpelius, kurie stipriai koreliuoja su kitais)

In [131]:
liver_patient = liver_patient.drop(['Direct_Bilirubin','Alamine_Aminotransferase','Albumin_and_Globulin_Ratio'],axis=1)
liver_patient.head()

,Age,Gender,Total_Bilirubin,Alkaline_Phosphotase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Liver_Disease,Globulin,Indirect_Bilirubin,Protiens_Ratio
0,65,0,0.7,187,18,6.8,3.3,1,3.666667,0.6,2.060606
1,62,1,10.9,699,100,7.5,3.2,1,4.324324,5.4,2.343750
2,62,1,7.3,490,68,7.0,3.3,1,3.707865,3.2,2.121212
3,58,1,1.0,182,20,6.8,3.4,1,3.400000,0.6,2.000000
4,72,1,3.9,195,59,7.3,2.4,1,6.000000,1.9,3.041667


In [132]:
np.random.seed(1)
mask = np.random.random(liver_patient.shape[0]) < 0.7

In [133]:
model3 = LogisticRegression()

model3.fit(liver_patient[mask].drop('Liver_Disease', axis=1), liver_patient[mask]['Liver_Disease'])
pred3 = model3.predict(liver_patient[~mask].drop('Liver_Disease', axis=1))
prob3 = model3.predict_proba(liver_patient[~mask].drop('Liver_Disease', axis=1))

print('Log-loss = {0:.02f}'.format(
    log_loss(liver_patient[~mask]['Liver_Disease'], prob3)))
print('Hit rate = {0:.02%}'.format(
    (liver_patient[~mask]['Liver_Disease'] == pred3).mean()))

Log-loss = 0.53
Hit rate = 70.22%


C:\Users\nasev\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning:

Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.



In [134]:
# suskaičiuojame Cofusion matrix
pd.DataFrame(confusion_matrix(liver_patient[~mask]['Liver_Disease'], pred3),
             columns=model3.classes_, index=model3.classes_)

,1,2
1,115,8
2,45,10
